# 引入所需要模块

In [2]:
import os
import random
import pandas as pd
import numpy as np
from PIL import Image

# 加载数据集

In [3]:
root_dir=os.path.abspath('.')
train=pd.read_csv('train.csv')
test=pd.read_csv('test.csv')

In [4]:
#随机读取图片
i=random.choice(train.index)
img_name=train.ID[i]
print(img_name)
img=Image.open(os.path.join('Train',img_name))
img.show()
print(train.Class[i])

24160.jpg
YOUNG


# 格式化图片尺寸和将数据存到numpy数组

In [5]:
temp=[]
for img_name in train.ID:
    img_path=os.path.join('Train',img_name)
    img=Image.open(img_path)
    img=img.resize((32,32))
    array=np.array(img)
    temp.append(array.astype('float32'))
train_x=np.stack(temp)
print(train_x.shape)
print(train_x.ndim)

(19906, 32, 32, 3)
4


In [6]:
temp=[]
for img_name in test.ID:
    img_path=os.path.join('Test',img_name)
    img=Image.open(img_path)
    img=img.resize((32,32))
    array=np.array(img)
    temp.append(array.astype('float32'))
test_x=np.stack(temp)
print(test_x.shape)

(6636, 32, 32, 3)


In [8]:
train_x = train_x / 255.
test_x = test_x / 255.

In [9]:
train.Class.value_counts(normalize=True)

MIDDLE    0.542751
YOUNG     0.336883
OLD       0.120366
Name: Class, dtype: float64

In [10]:
test['Class'] = 'MIDDLE'
test.to_csv('sub01.csv', index=False)

# 创建模型

In [11]:
import keras
from sklearn.preprocessing import LabelEncoder
lb=LabelEncoder()
train_y=lb.fit_transform(train.Class)
train_y=keras.utils.np_utils.to_categorical(train_y)
print(train_y)
print(train_y.shape)

Using TensorFlow backend.


[[ 1.  0.  0.]
 [ 0.  0.  1.]
 [ 1.  0.  0.]
 ..., 
 [ 1.  0.  0.]
 [ 1.  0.  0.]
 [ 1.  0.  0.]]
(19906, 3)


In [12]:
#构建神经网络
input_num_units=(32,32,3)
hidden_num_units=500
output_num_units=3
epochs=5
batch_size=128

In [49]:
from keras.models import Sequential
from keras.layers import Dense,Flatten,InputLayer
model=Sequential({
    InputLayer(input_shape=input_num_units),
    Flatten(),
    Dense(units=hidden_num_units,activation='relu'),
    Dense(units=output_num_units,activation='softmax')
})
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_37 (InputLayer)        (None, 32, 32, 3)         0         
_________________________________________________________________
dense_73 (Dense)             (None, 32, 32, 500)       2000      
_________________________________________________________________
flatten_37 (Flatten)         (None, 512000)            0         
_________________________________________________________________
dense_74 (Dense)             (None, 3)                 1536003   
Total params: 1,538,003
Trainable params: 1,538,003
Non-trainable params: 0
_________________________________________________________________


# 编译模型

In [ ]:
# model.compile(optimizer='sgd',loss='categorical_crossentropy',metrics=['accuracy'])
model.compile(optimizer='sgd',loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(train_x,train_y,batch_size=batch_size,epochs=epochs,verbose=1)

Epoch 1/5
15488/19906 [======================>.......] - ETA: 39s - loss: 0.9808 - acc: 0.5393

In [93]:
model.fit(train_x, train_y, batch_size=batch_size,epochs=epochs,verbose=1, validation_split=0.2)

Train on 15924 samples, validate on 3982 samples
Epoch 1/5
15924/15924 [==============================] - 6s - loss: 7.3748 - acc: 0.5425 - val_loss: 7.3507 - val_acc: 0.5439
Epoch 2/5
15924/15924 [==============================] - 7s - loss: 7.3748 - acc: 0.5425 - val_loss: 7.3507 - val_acc: 0.5439
Epoch 3/5
15924/15924 [==============================] - 6s - loss: 7.3748 - acc: 0.5425 - val_loss: 7.3507 - val_acc: 0.5439
Epoch 4/5
15924/15924 [==============================] - 7s - loss: 7.3748 - acc: 0.5425 - val_loss: 7.3507 - val_acc: 0.5439
Epoch 5/5
15924/15924 [==============================] - 6s - loss: 7.3748 - acc: 0.5425 - val_loss: 7.3507 - val_acc: 0.5439


# 输出结果

In [96]:
pred=model.predict_classes(test_x)
pred=lb.inverse_transform(pred)
print(pred)
test['Class']=pred
test.to_csv('sub02.csv',index=False)

6528/6636 [============================>.] - ETA: 0s['MIDDLE' 'MIDDLE' 'MIDDLE' ..., 'MIDDLE' 'MIDDLE' 'MIDDLE']


In [100]:
i = random.choice(train.index)
img_name = train.ID[i]

img=Image.open(os.path.join('Train',img_name))
img.show()
pred = model.predict_classes(train_x)
print('Original:', train.Class[i], 'Predicted:', lb.inverse_transform(pred[i]))

19906/19906 [==============================] - 3s     
Original: MIDDLE Predicted: MIDDLE
